In [46]:
import pandas as pd
import numpy as np

In [5]:
#데이터 불러오기 및 크기 확인
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print("훈련용 데이터 크기:", train_data.shape)
print("테스트용 데이터 크기:", test_data.shape)

train_data.columns

훈련용 데이터 크기: (17480, 16)
테스트용 데이터 크기: (15081, 15)


Index(['id', 'age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country',
       'target'],
      dtype='object')

In [6]:
#NA 처리

#국가 na 처리
train_modified=train_data[train_data['native.country'].isna()!=True].copy()
test_modified=test_data[test_data['native.country'].isna()!=True].copy()

In [10]:
#workclass & occupation na 처리

type1= train_modified['workclass'].isna()
type2= train_modified['occupation'].isna()

type3= train_modified['capital.gain']<50000
type4=train_modified['target']==1

train_modified.loc[type1&type2,'workclass']=train_modified.loc[type1&type2,'workclass'].replace(np.nan,'no-working')
train_modified.loc[type1&type2,'occupation']=train_modified.loc[type1&type2,'occupation'].replace(np.nan,'baeksu')
train_modified.drop(train_modified[type1&type2&type3&type4].index , axis=0,inplace=True)
# two_na 지만, 소득이 상위권중 capital_gain도 낮은 경우는 제거.


train_modified=train_modified.dropna()#남은 na제거

In [11]:
#education grouping


edu_group1=[16,15]
edu_group2=[15,14,13,11,12,10,9]
edu_group3=[4,6,8,5,3,7,2,1]

train_modified['education1']=0
train_modified['education2']=0
train_modified['education3']=0

test_modified['education1']=0
test_modified['education2']=0
test_modified['education3']=0


train_modified.loc[train_modified['education.num'].isin(edu_group1),'education1'] = 1
train_modified.loc[train_modified['education.num'].isin(edu_group2),'education2'] = 1
train_modified.loc[train_modified['education.num'].isin(edu_group3),'education3'] = 1

#education num 자체는 순위형 데이터라 삭제안함.
test_modified.loc[test_modified['education.num'].isin(edu_group1),'education1'] = 1
test_modified.loc[test_modified['education.num'].isin(edu_group2),'education2'] = 1
test_modified.loc[test_modified['education.num'].isin(edu_group3),'education3'] = 1

In [12]:
#education 제거
train_modified.drop('education',axis=1,inplace=True)
test_modified.drop('education',axis=1,inplace=True)

#relationship drop
train_modified.drop('relationship',axis=1,inplace=True)
test_modified.drop('relationship',axis=1,inplace=True)

In [14]:
#국가 grouping
country_group1=[]
country_group2=[]
country_group3=[]
country_group4=[]

for country in train_modified['native.country'].unique():
    country_data=train_modified[train_modified['native.country']==country]['target']
    probs=sum(country_data)/country_data.count()
    if probs >=0.5:
        country_group1.append(country)
    elif probs < 0.5 and probs >=0.3:
        country_group2.append(country)
    elif probs < 0.3 and probs >=0.1:
        country_group3.append(country)
    else:
        country_group4.append(country)
    #print(country,probs)

print(country_group1)
print(country_group2)
print(country_group3)
print(country_group4)

train_modified['country_group1']=0
train_modified['country_group2']=0
train_modified['country_group3']=0
train_modified['country_group4']=0


test_modified['country_group1']=0
test_modified['country_group2']=0
test_modified['country_group3']=0
test_modified['country_group4']=0


train_modified.loc[train_modified['native.country'].isin(country_group1),'country_group1'] = 1
train_modified.loc[train_modified['native.country'].isin(country_group2),'country_group2'] = 1
train_modified.loc[train_modified['native.country'].isin(country_group3),'country_group3'] = 1
train_modified.loc[train_modified['native.country'].isin(country_group4),'country_group4'] = 1

test_modified.loc[test_modified['native.country'].isin(country_group1),'country_group1'] = 1
test_modified.loc[test_modified['native.country'].isin(country_group2),'country_group2'] = 1
test_modified.loc[test_modified['native.country'].isin(country_group3),'country_group3'] = 1
test_modified.loc[test_modified['native.country'].isin(country_group4),'country_group4'] = 1

train_modified.drop('native.country',axis=1,inplace=True)
test_modified.drop('native.country',axis=1,inplace=True)

['Iran']
['Greece', 'Canada', 'India', 'England', 'Yugoslavia', 'Japan']
['United-States', 'Poland', 'Ireland', 'El-Salvador', 'Philippines', 'Italy', 'South', 'Cuba', 'Laos', 'Hong', 'Germany', 'Ecuador', 'France', 'Taiwan', 'China', 'Honduras', 'Peru', 'Hungary', 'Cambodia', 'Trinadad&Tobago', 'Thailand']
['Mexico', 'Guatemala', 'Dominican-Republic', 'Portugal', 'Jamaica', 'Columbia', 'Vietnam', 'Haiti', 'Puerto-Rico', 'Outlying-US(Guam-USVI-etc)', 'Nicaragua', 'Scotland', 'Holand-Netherlands']


In [15]:
#age 가공
train_modified['age']=np.log(train_modified['age'])# log로 
train_modified['age_group']=0

test_modified['age']=np.log(test_modified['age'])# log로 
test_modified['age_group']=0


train_modified.loc[(train_modified['age']>3.25)& (train_modified['age']<4.1),'age_group']=1
test_modified.loc[(test_modified['age']>3.25)& (test_modified['age']<4.1),'age_group']=1

In [17]:
train_modified.to_csv('train_cat_modified.csv',index=False)
test_modified.to_csv('test_cat_modified.csv',index=False)

In [73]:
train_modified = pd.read_csv('train_cat_modified.csv')
test_modified = pd.read_csv('test_cat_modified.csv')

,id,age,workclass,fnlwgt,education.num,marital.status,occupation,race,sex,capital.gain,...,hours.per.week,target,education1,education2,education3,country_group1,country_group2,country_group3,country_group4,age_group
0,0,3.465736,Private,309513,12,Married-civ-spouse,Craft-repair,White,Male,0,...,40,0,0,1,0,0,0,1,0,1
1,1,3.496508,Private,205469,10,Married-civ-spouse,Exec-managerial,White,Male,0,...,40,1,0,1,0,0,0,1,0,1
2,2,3.828641,Private,149949,10,Married-civ-spouse,Craft-repair,White,Male,0,...,40,0,0,1,0,0,0,1,0,1
3,3,3.135494,Private,193090,13,Never-married,Adm-clerical,White,Female,0,...,30,0,0,1,0,0,0,1,0,0
4,4,4.007333,Private,60193,9,Divorced,Adm-clerical,White,Female,0,...,40,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17470,17470,4.094345,no-working,134152,5,Divorced,baeksu,Black,Male,0,...,35,0,0,0,1,0,0,1,0,1
17472,17472,4.406719,no-working,403910,9,Never-married,baeksu,White,Male,0,...,3,0,0,1,0,0,0,1,0,0
17476,17476,3.401197,no-working,33811,13,Never-married,baeksu,Asian-Pac-Islander,Female,0,...,99,0,0,1,0,0,0,1,0,1
17478,17478,3.713572,no-working,202822,9,Separated,baeksu,Black,Female,0,...,32,0,0,1,0,0,0,1,0,1


In [19]:
#필요 라이브러리들 import

import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dropout

from sklearn.model_selection import train_test_split

In [74]:
x_train = train_modified.drop(['id', 'target'], axis=1)
y_train = train_modified['target']

test_data=test_modified.drop(['id'], axis=1)


print("훈련용 데이터:", x_train.shape)
print("테스트용 라벨:", y_train.shape)

훈련용 데이터: (16707, 19)
테스트용 라벨: (16707,)


In [75]:
#라벨인코딩을 하기 위함 dictionary map 생성 함수
def make_label_map(dataframe):
    label_maps = {}
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            label_map = {'unknown':0}
            for i, key in enumerate(dataframe[col].unique()):
                label_map[key] = i  #새로 등장하는 유니크 값들에 대해 1부터 1씩 증가시켜 키값을 부여해줍니다.
            label_maps[col] = label_map
    return label_maps

# 각 범주형 변수에 인코딩 값을 부여하는 함수
def label_encoder(dataframe, label_map):
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            dataframe[col] = dataframe[col].map(label_map[col])
            #dataframe[col] = dataframe[col].fillna(label_map[col]['unknown']) #혹시 모를 결측값은 unknown의 값(0)으로 채워줍니다.
    return dataframe

x_train = label_encoder(x_train, make_label_map(x_train))

#제출용 라벨링 데이터 변환
test_data = label_encoder(test_data, make_label_map(test_data))

In [76]:

X_train, X_test, Y_train,Y_test = train_test_split(x_train, y_train, random_state=22)

In [77]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train_centered = (X_train  - mean_vals) / std_val

mean_vals = np.mean(X_test, axis=0)
std_val = np.std(X_test)

X_test_centered = (X_test- mean_vals) / std_val

In [78]:
#원핫 인코딩 생성
y_train_onehot = tf.keras.utils.to_categorical(Y_train)
print(y_train_onehot[:3])

[[1. 0.]
 [0. 1.]
 [0. 1.]]


In [93]:
model = tf.keras.models.Sequential()

model.add(
    tf.keras.layers.Dense(
    units=100,
    input_dim=X_train_centered.shape[1],
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='relu'
    )
)

model.add(
    tf.keras.layers.Dense(
    units=50,
    input_dim=100,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='tanh'
    )
)
model.add(Dropout(0.5))


"""

model.add(
    tf.keras.layers.Dense(
    units=100,
    input_dim=150,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='tanh'
    )
)

model.add(
    tf.keras.layers.Dense(
    units=50,
    input_dim=100,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='tanh'
    )
)

model.add(
    tf.keras.layers.Dense(
    units=30,
    input_dim=50,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='tanh'
    )
)

model.add(
    tf.keras.layers.Dense(
    units=20,
    input_dim=30,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='tanh'
    )
)
"""

model.add(
    tf.keras.layers.Dense(
    units=y_train_onehot.shape[1],
    input_dim=50,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='softmax'
    )
)


In [94]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 100)               2000      
_________________________________________________________________
dense_13 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 102       
Total params: 7,152
Trainable params: 7,152
Non-trainable params: 0
_________________________________________________________________


In [95]:
model.compile(optimizer='adam',metrics=['accuracy'], loss="categorical_crossentropy")


In [96]:
history = model.fit(X_train_centered, y_train_onehot, batch_size=32, epochs=10, verbose=1, validation_split=0.1)

Epoch 1/10
353/353 [==============================] - 1s 3ms/step - loss: 0.3684 - accuracy: 0.8327 - val_loss: 0.3101 - val_accuracy: 0.8508
Epoch 2/10
353/353 [==============================] - 1s 3ms/step - loss: 0.3330 - accuracy: 0.8456 - val_loss: 0.3017 - val_accuracy: 0.8603
Epoch 3/10
353/353 [==============================] - 1s 3ms/step - loss: 0.3239 - accuracy: 0.8493 - val_loss: 0.2984 - val_accuracy: 0.8611
Epoch 4/10
353/353 [==============================] - 1s 2ms/step - loss: 0.3158 - accuracy: 0.8556 - val_loss: 0.2949 - val_accuracy: 0.8651
Epoch 5/10
353/353 [==============================] - 1s 3ms/step - loss: 0.3156 - accuracy: 0.8516 - val_loss: 0.2944 - val_accuracy: 0.8691
Epoch 6/10
353/353 [==============================] - 1s 3ms/step - loss: 0.3090 - accuracy: 0.8532 - val_loss: 0.2957 - val_accuracy: 0.8611
Epoch 7/10
353/353 [==============================] - 1s 3ms/step - loss: 0.3081 - accuracy: 0.8536 - val_loss: 0.2985 - val_accuracy: 0.8611
Epoch 

In [97]:
y_train_pred= model.predict_classes(X_train_centered, verbose=0)
correct_preds = np.sum(Y_train == y_train_pred, axis=0)

train_acc = correct_preds / Y_train.shape[0]

y_test_pred= model.predict_classes(X_test_centered, verbose=0)
correct_preds = np.sum(Y_test == y_test_pred, axis=0)

test_acc = correct_preds / Y_test.shape[0]

print("train 최종 결과", train_acc)
print("test 최종 결과", test_acc)

C:\Users\yong\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


train 최종 결과 0.8637669592976855
test 최종 결과 0.8587502992578405


C:\Users\yong\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [98]:
#훈련용 데이터 전부 사용
mean_vals = np.mean(x_train, axis=0)
std_val = np.std(x_train)

result_centered = (x_train  - mean_vals) / std_val

result_onehot = tf.keras.utils.to_categorical(y_train)

In [100]:
history = model.fit(result_centered, result_onehot, batch_size=48, epochs=20, verbose=1)

Epoch 1/20
349/349 [==============================] - 1s 2ms/step - loss: 0.2878 - accuracy: 0.8649
Epoch 2/20
349/349 [==============================] - 1s 2ms/step - loss: 0.2884 - accuracy: 0.8660
Epoch 3/20
349/349 [==============================] - 1s 2ms/step - loss: 0.2867 - accuracy: 0.8667
Epoch 4/20
349/349 [==============================] - 1s 2ms/step - loss: 0.2880 - accuracy: 0.8677
Epoch 5/20
349/349 [==============================] - 1s 2ms/step - loss: 0.2855 - accuracy: 0.8660
Epoch 6/20
349/349 [==============================] - 1s 2ms/step - loss: 0.2854 - accuracy: 0.8666
Epoch 7/20
349/349 [==============================] - 1s 2ms/step - loss: 0.2850 - accuracy: 0.8663
Epoch 8/20
349/349 [==============================] - 1s 2ms/step - loss: 0.2851 - accuracy: 0.8670
Epoch 9/20
349/349 [==============================] - 1s 2ms/step - loss: 0.2830 - accuracy: 0.8698
Epoch 10/20
349/349 [==============================] - 1s 2ms/step - loss: 0.2828 - accuracy: 0.8660

In [101]:
mean_vals = np.mean(test_data, axis=0)
std_val = np.std(test_data)

test_centered = (test_data  - mean_vals) / std_val
test_pred= model.predict_classes(test_centered, verbose=0)


C:\Users\yong\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [102]:
result = pd.DataFrame({'target': test_pred})
result.to_csv('day4_final.csv')